In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/refcoco.pth
/kaggle/input/reference.txt
/kaggle/input/dinov2_vitl14_pretrain.pth
/kaggle/input/sam_vit_h_4b8939.pth
/kaggle/input/sample_image.jpg
/kaggle/input/swint_only_sam_many2many.pth
/kaggle/input/Images-20250418T060633Z-001/Images/backpack_dog/01.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/backpack_dog/00.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/clock/01.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/clock/00.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/can/01.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/can/00.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/berry_bowl/01.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/berry_bowl/00.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/candle/01.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/candle/00.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/barn/01.jpg
/kaggle/input/Images-20250418T060633Z-001/Images/barn/00.jpg
/kaggle/input/Images-20250418T06

In [2]:
!pip install ftfy packaging regex tqdm torch torchvision huggingface_hub safetensors timm open_clip_torch fairscale==0.4.4 pycocoevalcap # requests filelock mmcv-full==1.3.12 mmsegmentation pycocotools==2.0.2 opencv-python==4.5.3.56

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForQuestionAnswering, BlipForConditionalGeneration
import torch
from PIL import Image
import requests
import torch.nn.functional as F

2025-04-20 15:03:56.110858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745161436.331384      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745161436.400102      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
image_path = "/kaggle/input/sample_image.jpg"
image = Image.open(image_path)

# 2. Visual QA

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
blip_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
image_2 = image.convert("RGB")
question = "Where is the dog present in the image?"

In [8]:
inputs_3 = blip_processor(image_2, question, return_tensors="pt").to(device)

with torch.no_grad():
    output_3 = blip_model.generate(**inputs_3)

answer_3 = blip_processor.decode(output_3[0], skip_special_tokens=True)
print("Question:", question)
print("Reply:", answer_3)

Question: Where is the dog present in the image?
Reply: in man ' s arms


In [9]:
question_2 = "Where is the man present in the image?"
inputs_3_2 = blip_processor(image_2, question_2, return_tensors="pt").to(device)

with torch.no_grad():
    output_3_2 = blip_model.generate(**inputs_3_2)

answer_3_2 = blip_processor.decode(output_3_2[0], skip_special_tokens=True)
print("Question:", question_2)
print("Reply:", answer_3_2)

Question: Where is the man present in the image?
Reply: living room
